In [ ]:
import matplotlib
#matplotlib.use("PDF")
import glob,time,scipy,pickle
from astropy.io import fits as pf
import scipy.interpolate
from scipy.special import erf
from scipy import stats
from pylab import *
from subprocess import call
from scipy.optimize import fmin_powell
from scipy.ndimage import gaussian_filter, convolve
topdir='/Users/cslage/Research/LSST/code/GUI/brighter_fatter/'
thedir=topdir+'notebooks/'
#sys.path.append(thedir)
#import forward_sky
datadir=topdir+'/20171128_002_spots_VBB60/'
kerneldir = datadir

configfile=topdir+'sextractor/default-array_dither.sex'
paramfile=topdir+'sextractor/default-array_dither.param'
maskdir=topdir+'sextractor/masks/'
%matplotlib inline


In [ ]:
# These files have been crosstalk subtracted
filelist=np.sort(glob.glob(datadir+'ITL-3800C-002_spot_spot_???_20171128??????_ct.fits'))
zfilelist = []
# Now select the files in best focus from earlier analysis
for file in filelist:
    seqno = int(file.split('_')[-3])
    if seqno in [228 + 20 * n for n in range(20)]:
        zfilelist.append(file)
print(len(zfilelist))
#print(zfilelist)

In [ ]:
def remove_overscan_xy(image,x_overscan_start,x_overscan_end,y_overscan_start,y_overscan_end):
    overscan=image[:y_overscan_start,x_overscan_start+1:x_overscan_end]
    image=image[:y_overscan_start,:x_overscan_start]
    finalimg=image-matrix(median(overscan,axis=1)).T*np.ones(shape(image)[1])
    return array(finalimg)

def make_reg_from_ldac(cat_ldac_file,text_tag):
    thecat=pf.getdata(cat_ldac_file,'LDAC_OBJECTS')
    f = open(cat_ldac_file+'.reg','w')
    for i in range(len(thecat)):
        xcoo,ycoo=thecat['XWIN_IMAGE'][i],thecat['YWIN_IMAGE'][i]
        r=thecat['A_IMAGE'][i]
        thetext=thecat[text_tag][i]
        f.write('circle '+str(xcoo)+' '+str(ycoo)+' '+str(r)+'#text="'+str(thetext)+'"\n')
    f.close()


In [ ]:
# This runs sextractor on the files to find the spots.  This only needs to be done once
for fitsfilename in zfilelist: 
    tfile1=time.time() 
    for i in range(16):
        tstart=time.time() 
        extname=pf.getheader(fitsfilename,i+1)['EXTNAME'] 
        img=pf.getdata(fitsfilename,extname) 
        overscansubimg=remove_overscan_xy(img,509,542,2000,2022) 
        # cut off the overscan 
        outname=fitsfilename[:-5]+extname+'.fits' 
        pf.writeto(outname,overscansubimg,clobber=True) 
        test=call(["sex",outname,"-c",configfile,"-CATALOG_NAME",outname+'.cat']) 
        make_reg_from_ldac(outname+'.cat','NUMBER') 
        tend=time.time() 
        print extname+" time: "+str(tend-tstart)[:4] 
        print "Time taken for file "+str(fitsfilename[-26:-23])+": "+str(time.time()-tfile1)

#### First get some spot data

In [ ]:
def remove_overscan_xy(image,x_overscan_start,x_overscan_end,y_overscan_start,y_overscan_end):
    overscan=image[:y_overscan_start,x_overscan_start+1:x_overscan_end]
    image=image[:y_overscan_start,:x_overscan_start]
    finalimg=image-matrix(median(overscan,axis=1)).T*np.ones(shape(image)[1])
    return array(finalimg)

class Array2dSet:
    def __init__(self,xmin,xmax,nx,ymin,ymax,ny,nstamps):
        # This packages up a set of nstamps postage stamp images,
        # each image of which is nx * ny pixels
        self.nx=nx
        self.ny=ny
        self.nstamps=nstamps

        self.xmin=xmin
        self.ymin=ymin
        
        self.xmax=xmax
        self.ymax=ymax
        
        self.dx=(xmax-xmin)/nx
        self.dy=(ymax-ymin)/ny
        
        self.x=linspace(xmin+self.dx/2,xmax-self.dx/2,nx)
        self.y=linspace(ymin+self.dy/2,ymax-self.dy/2,ny)

        self.data=zeros([nx,ny,nstamps])
        self.xoffset=zeros([nstamps])
        self.yoffset=zeros([nstamps])
        self.imax=zeros([nstamps])


def BuildSpotList(fitsfilename, segmentnumber, numspots, nx, ny, minsize, maxsize):
    stampxmin = -(int(nx/2)+0.5)
    stampxmax = -stampxmin
    stampymin = -(int(ny/2)+0.5)
    stampymax = -stampymin
    xcoomin = 50
    xcoomax = 450
    ycoomin = 1400
    ycoomax = 1900
    spotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,numspots)
    hdr=pf.getheader(fitsfilename,segmentnumber)
    extname = hdr['EXTNAME']
    img=pf.getdata(fitsfilename,extname) 
    overscansubimg=remove_overscan_xy(img,509,542,2000,2022) 
    catname=fitsfilename[:-5]+extname+'.fits.cat.reg' 
    catfile = open(catname,'r')
    catlines = catfile.readlines()
    #print(catlines)
    catfile.close()
    n=0
    for line in catlines:
        try:
            size = float(line.split()[3].split('#')[0])
            if size < minsize or size > maxsize:
                continue
            xcoord = float(line.split()[1])
            ycoord = float(line.split()[2])
            if xcoord < xcoomin or xcoord > xcoomax or ycoord < ycoomin or ycoord > ycoomax:
                continue
            xint = int(xcoord-0.5)
            yint = int(ycoord-0.5)
            xmin = xint - int(nx/2)
            xmax = xint + int(nx/2) + 1
            ymin = yint - int(ny/2)
            ymax = yint + int(ny/2) + 1
            stamp = overscansubimg[ymin:ymax, xmin:xmax]
           
            xsum = 0.0
            ysum = 0.0
            datasum = 0.0
             
            for i in range(nx):
                for j in range(ny):
                    spotlist.data[i,j,n] = float(stamp[j,i])                    
                    ysum += spotlist.y[j] * spotlist.data[i,j,n]
                    xsum += spotlist.x[i] * spotlist.data[i,j,n]
                    datasum += spotlist.data[i,j,n]
            xoff = xsum / datasum
            yoff = ysum / datasum
            spotlist.xoffset[n] = xoff
            spotlist.yoffset[n] = yoff
            #spotlist.xoffset[n] = xcoord - xint - 1.0
            #spotlist.yoffset[n] = ycoord - yint - 1.0     
            #print "Spot = %d, calc_off = (%f,%f), sex_off = (%f,%f)"%(n,xoff,yoff,xcoord-xint-1.0,ycoord-yint-1.0)
            n += 1
            if n == numspots:
                return spotlist
        except:
            continue
    # Reaching this point means we found less spots than requested.
    newspotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,n)
    newspotlist.xoffset = spotlist.xoffset[0:n]
    newspotlist.yoffset = spotlist.yoffset[0:n]
    newspotlist.data = spotlist.data[:,:,0:n]
    del spotlist
    return newspotlist


def Area(xl, xh, yl, yh, sigmax, sigmay, Imax):
    # Calculates how much of a 2D Gaussian falls within a rectangular box
    ssigx = sqrt(2) * sigmax
    ssigy = sqrt(2) * sigmay    
    I = (erf(xh/ssigx)-erf(xl/ssigx))*(erf(yh/ssigy)-erf(yl/ssigy))
    return Imax * I / 4.0


def FOM(params):
    [sigmax, sigmay, sky] = params
    result = forward_sky.forward_sky(spotlist,sigmax,sigmay,sky)
    return result


In [ ]:
for segmentnumber in range(1,17):
    hdr=pf.getheader(zfilelist[0],segmentnumber)
    extname = hdr['EXTNAME']
    print segmentnumber, extname


In [ ]:
#Check the spotlist building and look at a typical spot
nx = 11
ny = 11
numspots =400
spotnum = 2
segmentnumber = 13

fitsfilename = zfilelist[1]
print(fitsfilename)
    
spotlist = BuildSpotList(fitsfilename, segmentnumber, numspots, nx, ny, 0.7, 1.4)
print("nstamps = %d"%spotlist.nstamps)
#args = ()
#param0 = [1.0, 1.0, 1.0]
#Result = fmin_powell(FOM, param0, args)
#OldResult = Result
#print "Result = ", Result

spot = 4.4 * spotlist.data[:,:,spotnum] # Convert to electrons
print("xoff = %f, yoff = %f"%(spotlist.xoffset[spotnum],spotlist.yoffset[spotnum]))

imshow(spot, interpolation='nearest')
colorbar()


#### Now extract the kernel

In [ ]:
for i in range(10):
    peak = spotlist.data[:,:,i].max()
    tot = spotlist.data[:,:,i].sum()
    ratio = tot/peak
    print(peak, tot, ratio)


In [ ]:
class Array2d:
    def __init__(self,xmin,xmax,nx,ymin,ymax,ny):
        # Each image is nx * ny pixels
        self.nx=nx
        self.ny=ny

        self.xmin=xmin
        self.ymin=ymin
        
        self.xmax=xmax
        self.ymax=ymax
        
        self.dx=(xmax-xmin)/nx
        self.dy=(ymax-ymin)/ny
        
        self.x=linspace(xmin+self.dx/2,xmax-self.dx/2,nx)
        self.y=linspace(ymin+self.dy/2,ymax-self.dy/2,ny)

        self.cov=zeros([nx,ny])
        self.kernel=zeros([nx,ny])       
        
    def PrintKernel(self, filename):
        file = open(filename, 'w')
        line = 'X\tY\tCovariance\t\t\tKernel\n'
        file.write(line)
        for i in range(self.nx):
            for j in range(self.ny):
                line = '%d\t%d\t%0.12e\t%0.12e\n'%(i,j,self.cov[i,j],self.kernel[i,j])
                file.write(line)
        file.close()
        return

def ReadConfigFile(filename):
    # This reads the Poisson simulator config file for
    # the settings that were run
    # and returns a dictionary with the values

    with open(filename,'r') as file:
        lines=file.readlines()
    lines = [ l.strip() for l in lines ]
    lines = [ l.split() for l in lines if len(l) > 0 and l[0] != '#' ]
    for line in lines:
        if line[1] != '=':
            print "Check line: ",line
            raise IOError("Error reading config file %s"%filename)
    config = {}
    for line in lines:
        try:
            # First get the ordered pairs
            config.update({line[0]:[eval(line[2]), eval(line[3])]})
        except:
            try:
                # Then get the numeric values
                config.update({line[0]:eval(line[2])})
            except:
                try:
                    # Last, get the strings
                    config.update({line[0]:str(line[2])})
                except:
                    pass
    return config


def ReadAreaFile(filename, arr, Area_0, NxCenter, NyCenter):
    # This reads the simulated pixel area file
    # and returns an array with the covariances
    file = open(filename, 'r')
    lines = file.readlines()
    file.close()
    lines.remove(lines[0]) # Strip the title line    
    for line in lines:
        items = line.split()
        i = int(items[0])
        j = int(items[1])
        area = float(items[2])
        ii = i + (arr.nx - 1)/2 - NxCenter 
        jj = j + (arr.ny - 1)/2 - NyCenter 
        arr.cov[ii,jj] += (area - Area_0) / Area_0
    return arr

def ReadMeasurementFile(filename, arr):
    # This reads the measured correlation data file
    # and returns an array with the covariances
    file = open(filename, 'r')
    lines = file.readlines()
    file.close()
    lines.remove(lines[0]) # Strip the title line
    for i in range(3):
        lines.remove(lines[-1]) # Strip the last three lines

    for line in lines:
        items = line.split()
        i = int(items[0])
        j = int(items[1])
        cov = float(items[2])
        if (i > 0 or j > 0) and cov < 0.0:
            cov = 0.0
        for xsign in [-1,1]:
            ii = i * xsign + (arr.nx - 1)/2
            for ysign in [-1,1]:
                jj = j * ysign + (arr.ny - 1)/2
                arr.cov[ii,jj] = cov # Baseline
    return arr


def SOR(arr, w):
    hsquared =  arr.dx * arr.dy;
    omw = 1.0 - w;
    w4 = w / 4.0
    for i in range(1, arr.nx-1):
        for j in range(1, arr.ny-1):
            arr.kernel[i,j] = omw * arr.kernel[i,j] + w4 * (arr.kernel[i-1,j] + arr.kernel[i+1,j] + arr.kernel[i,j-1] + arr.kernel[i,j+1] + hsquared * arr.cov[i,j])
    return arr
  

In [ ]:
# First, read the .cfg file
configfile = kerneldir+'pixel.cfg'
run = 0
ConfigData = ReadConfigFile(configfile)
outputfilebase = ConfigData["outputfilebase"]
outputfiledir = ConfigData["outputfiledir"]
Nx = ConfigData["PixelBoundaryNx"]
Ny = ConfigData["PixelBoundaryNy"]
XCenter = ConfigData["FilledPixelCoords_0_0"][0]
YCenter = ConfigData["FilledPixelCoords_0_0"][1]
PixelSizeX = ConfigData["PixelSizeX"]
PixelSizeY = ConfigData["PixelSizeY"]
NxCenter = int((XCenter - ConfigData["PixelBoundaryLowerLeft"][0]) / PixelSizeX)
NyCenter = int((YCenter - ConfigData["PixelBoundaryLowerLeft"][1]) / PixelSizeY)
NumElec = ConfigData["CollectedCharge_0_0"]
NewNx = NewNy = 21
NewNxCenter = (NewNx - 1) / 2
NewNyCenter = (NewNy - 1) / 2

filename = kerneldir + 'corr_meas_model_central_csteps_7_07jun18.txt'
kernel = Array2d(0,NewNx,NewNx,0,NewNy,NewNy)
kernel = ReadMeasurementFile(filename, kernel)
kernel.cov /= NumElec

w = 1.9
for n in range(2000):
    kernel = SOR(kernel,w)

figure()
suptitle('B-F Kernel extracted from 3800 Flats')
subplots_adjust(wspace=0.8,hspace=0.5)
subplot(2,3,1)
title("Covariances(*1E7)",fontsize=9)
imshow(kernel.cov*1E7, interpolation='nearest')
colorbar(fraction=0.046)
subplot(2,3,2)
title("Covariances(*1E7) - X-Slice",fontsize=9)
plot(kernel.x,kernel.cov[:,NewNyCenter]*1E7)
subplot(2,3,3)
title("Covariances(*1E7) - Y-Slice",fontsize=9)
plot(kernel.y,kernel.cov[NewNxCenter,:]*1E7)
subplot(2,3,4)
title("Kernel(*1E7)",fontsize=9)
imshow(kernel.kernel*1E7, interpolation='nearest')
colorbar(fraction=0.046)
subplot(2,3,5)
title("Kernel(*1E7) - X-Slice",fontsize=9)
plot(kernel.x,kernel.kernel[:,NewNyCenter]*1E7)
subplot(2,3,6)
title("Kernel(*1E7) - Y-Slice",fontsize=9)
plot(kernel.y,kernel.kernel[NewNxCenter,:]*1E7)

savefig(kerneldir+"Kernel_Plot_Model_Central_CSteps_7_07Jun18.pdf")
kernel.PrintKernel(datadir+'kernel_model_central_csteps_7_07jun18.txt')

pickle_out = open(datadir+"kernel_model_central_csteps_7_07jun18.pickle","wb")
pickle.dump(kernel, pickle_out)
pickle_out.close()

In [ ]:
# What does the convolution of the kernel with the spot look like?
spot_kernel=convolve(spot,kernel.kernel,mode='constant',cval=0.0)
imshow(spot_kernel, interpolation='nearest')
colorbar()
show()

In [ ]:
# Now implement the correction
# Trying 4th order corrections
def Dx(image):
    dx = zeros(image.shape)
    for i in range(2,image.shape[0]-2):
        for j in range(image.shape[1]):
            dx[i,j] = (-image[i+2,j] + 8.0 * image[i+1,j] - 8.0 * image[i-1,j] + image[i-2,j])/12.0
    return dx

def Dy(image):
    dy = zeros(image.shape)
    for i in range(image.shape[0]):
        for j in range(2,image.shape[1]-2):
            dy[i,j] = (-image[i,j+2] + 8.0 * image[i,j+1] - 8.0 * image[i,j-1] + image[i,j-2])/12.0
    return dy

def D2(image):
    d2 = zeros(image.shape)
    for i in range(1,image.shape[0]-1):
        for j in range(1,image.shape[1]-1):
            d2[i,j] = (4.0 * (image[i+1,j] + image[i-1,j] + image[i,j+1] + image[i,j-1]) + \
                       (image[i+1,j+1] + image[i+1,j-1] + image[i-1,j+1] + image[i-1,j-1]) - \
                       20.0 * image[i,j]) / 6.0
    return d2

def Correction(data, kernel):
    int_term = convolve(data,kernel,mode='constant',cval=0.0)
    term1 = Dx(data) * Dx(int_term) + Dy(data)* Dy(int_term)
    term2 = data * D2(int_term)
    correction = (term1 + term2) / 2.0
    #print data.sum(), kernel.sum(), int_term.sum(), correction.sum()
    return correction



In [ ]:
# Now implement the correction - This is second order
# This hasn't worked as well, so is included just for completeness
"""
def Dx(image):
    dx = zeros(image.shape)
    for i in range(1,image.shape[0]-1):
        for j in range(image.shape[1]):
            dx[i,j] = (image[i+1,j] - image[i-1,j])/2.0
    return dx

def Dy(image):
    dy = zeros(image.shape)
    for i in range(image.shape[0]):
        for j in range(1,image.shape[1]-1):
            dy[i,j] = (image[i,j+1] - image[i,j-1])/2.0
    return dy

def D2(image):
    d2 = zeros(image.shape)
    for i in range(1,image.shape[0]-1):
        for j in range(1,image.shape[1]-1):
            d2[i,j] = (image[i+1,j] + image[i-1,j] + image[i,j+1] + image[i,j-1] - 4.0 * image[i,j])
    return d2

def Correction(data, kernel):
    int_term = convolve(data,kernel,mode='constant',cval=0.0)
    term1 = Dx(data) * Dx(int_term) + Dy(data)* Dy(int_term)
    term2 = data * D2(int_term)
    correction = (term1 + term2) / 2.0
    return correction

correction = Correction(spot, kernel.kernel)
imshow(correction, interpolation ='nearest')
colorbar()
show()
"""

In [ ]:
# Now try it on a single spot
newspot = copy(spot)
correction = zeros(spot.shape)
diff = 100.0
# Iterate to convergence
while diff > 1.0E-6:
    lastcorrection = copy(correction)
    correction = Correction(newspot, kernel.kernel)
    newspot = spot + correction
    diff = ((lastcorrection - correction) * (lastcorrection - correction)).sum()
    print "Diff = %f"%diff

print correction.max(), correction.min()
figure(figsize=(16,8))
subplots_adjust(wspace=0.5)
subplot(2,3,1)
title("X")
plot(spotlist.x, spot[:,5], label='Spot')
plot(spotlist.x, newspot[:,5], label='NewSpot')
plot(spotlist.x, 20*correction[:,5], label='20X Correction')
subplot(2,3,2)
title("Y")
plot(spotlist.y, spot[5,:], label='Spot')
plot(spotlist.y, newspot[5,:], label='NewSpot')
plot(spotlist.y, 20*correction[5,:], label='20X Correction')
legend(loc='center right', bbox_to_anchor=(2.0, 0.5))
subplot(2,3,4)
title("Spot")
imshow(spot, interpolation = 'nearest')
subplot(2,3,5)
title("NewSpot")
imshow(newspot, interpolation = 'nearest')
subplot(2,3,6)
title("Correction")
imshow(correction, interpolation = 'nearest')
colorbar()
#show()
savefig(datadir+"Coulton_Correction_Model_Central_CSteps_7_07Jun18.pdf")

In [ ]:
# test the correction implementation on a single spotlist
figure(figsize=(16,8))
subplots_adjust(wspace=0.5)

gain = 4.40
for spot in range(spotlist.nstamps):
    newspot = copy(gain * spotlist.data[:,:,spot])
    correction = zeros(newspot.shape)
    diff = 100.0
    numsteps = 0
    # Iterate to convergence
    while diff > 1.0E-6:
        lastcorrection = copy(correction)
        correction = Correction(newspot, kernel.kernel)
        newspot = gain * spotlist.data[:,:,spot] + correction
        diff = ((lastcorrection - correction) * (lastcorrection - correction)).sum()
        #print "Diff = %f"%diff
        numsteps += 1
    if spot % 100 == 0:
        print "Finished spot %d, Numsteps to converge = %d, Correction Sum = %f"\
        %(spot,numsteps, correction.sum()) 
    
    if spot == spotlist.nstamps - 1:  # Plot the last one, with correction
        subplot(2,3,1)
        title("X")
        plot(spotlist.x, spotlist.data[:,5,-1] * gain, label='Spot')
        plot(spotlist.x, newspot[:,5], label='NewSpot')
        plot(spotlist.x, 20*correction[:,5], label='20X Correction')
        subplot(2,3,2)
        title("Y")
        plot(spotlist.y, spotlist.data[5,:,-1] * gain, label='Spot')
        plot(spotlist.y, newspot[5,:], label='NewSpot')
        plot(spotlist.y, 20*correction[5,:], label='20X Correction')
        legend(loc='center right', bbox_to_anchor=(2.0, 0.5))
        subplot(2,3,4)
        title("Spot")
        imshow(spotlist.data[:,:,-1] * gain, interpolation = 'nearest')
        subplot(2,3,5)
        title("NewSpot")
        imshow(newspot, interpolation = 'nearest')
        subplot(2,3,6)
        title("Correction")
        imshow(correction, interpolation = 'nearest')
        colorbar()

    for i in range(spotlist.nx):
        for j in range(spotlist.ny):
            spotlist.data[i,j,spot] = newspot[i,j] / gain

args = ()#(spotlist)
param0 = [1.0, 1.0, 1.0]
Result = fmin_powell(FOM, param0, args)
print "Old Sigmas = ", OldResult,"New Sigmas = ", Result




In [ ]:
# Now let's try it with a full list of spots on two segments
import gc
nx = 11
ny = 11
numspots = 400
numfiles = len(zfilelist)
sigmaxs = zeros([1,numfiles])
sigmays = zeros([1,numfiles])
imaxs = zeros([1,numfiles])
totelectrons = zeros([1,numfiles])
corr_sigmaxs = zeros([1,numfiles])
corr_sigmays = zeros([1,numfiles])
corr_imaxs = zeros([1,numfiles])
corr_totelectrons = zeros([1,numfiles])

gains = [4.40]

for i, segmentnumber in enumerate([1]):
    gain = gains[i]

    for j,fitsfilename in enumerate(zfilelist):

        # First run the forward modeling on the uncorrected spotlist
        param0 = [1.0, 1.0, 1.0]
        spotlist = BuildSpotList(fitsfilename, segmentnumber, numspots, nx, ny,0.7,2.0)
        print "nstamps = %d"%spotlist.nstamps
        args = ()
        Result = fmin_powell(FOM, param0, args)

        print fitsfilename
        imax = spotlist.imax.mean()
        ADU_correction = Area(-0.5,0.5,-0.5,0.5,Result[0],Result[1],1.0)
        tote = spotlist.data.sum() * gain / spotlist.nstamps # Total electrons in the stamp
        print "Uncorrected:",Result, imax, tote
        imaxs[i,j] = imax * ADU_correction * gain
        sigmaxs[i,j] = abs(Result[0])
        sigmays[i,j] = abs(Result[1])
        totelectrons[i,j] = tote

        # Now apply the correction and recalculate the forward modeling
        for spot in range(spotlist.nstamps):
            newspot = copy(gain * spotlist.data[:,:,spot])
            correction = zeros(newspot.shape)
            diff = 100.0
            numsteps = 0
            # Iterate to convergence
            while diff > 1.0E-6:
                lastcorrection = copy(correction)
                correction = Correction(newspot, kernel.kernel) 
                newspot = gain * spotlist.data[:,:,spot] + correction
                diff = ((lastcorrection - correction) * (lastcorrection - correction)).sum()
                #print "Diff = %f"%diff
                numsteps += 1
            for ii in range(spotlist.nx):
                for jj in range(spotlist.ny):
                    spotlist.data[ii,jj,spot] = newspot[ii,jj] / gain
            if spot % 100 == 0:
                print "Finished spot %d, Numsteps to converge = %d"%(spot, numsteps)
        args = ()
        param0 = [1.0, 1.0, 1.0]
        Result = fmin_powell(FOM, param0, args)
        imax = spotlist.imax.mean()
        ADU_correction = Area(-0.5,0.5,-0.5,0.5,Result[0],Result[1],1.0)
        tote = spotlist.data.sum() * gain / spotlist.nstamps # Total electrons in the stamp
        print "Corrected",Result, imax, tote
        corr_imaxs[i,j] = imax * ADU_correction * gain
        corr_sigmaxs[i,j] = abs(Result[0])
        corr_sigmays[i,j] = abs(Result[1])
        corr_totelectrons[i,j] = tote
        del spotlist
        gc.collect()


In [ ]:
# Now plot the result
rcParams.update({'font.size':10})
# The following masks out bad fits files of undetermined cause, probably due to the shutter sticking
mask = []

for i, segmentnumber in enumerate([1]):
    hdr=pf.getheader(zfilelist[0],segmentnumber)
    extname = hdr['EXTNAME']

    figure()
    title("Brighter-Fatter - 30 micron Spots - %s"%extname)
    # First plot the uncorrected data
    scatter(delete(imaxs[i,:],mask), delete(sigmaxs[i,:],mask), color = 'green', lw = 2, label = 'Sigma-x')
    scatter(delete(imaxs[i,:],mask), delete(sigmays[i,:],mask), color = 'red', lw = 2, label = 'Sigma-y')

    slope, intercept, r_value, p_value, std_err = stats.linregress(imaxs[i,:],sigmaxs[i,:])
    xplot=linspace(-5000.0,300000.0,100)
    yplot = slope * xplot + intercept
    plot(xplot, yplot, color='green', lw = 2, ls = '--')
    tslope = slope * 100.0 * 50000.0
    text(10000.0,1.09,"X Slope = %.2f %% per 50K e-, Intercept = %.3f"%(tslope,intercept), fontsize=10)

    slope, intercept, r_value, p_value, std_err = stats.linregress(imaxs[i,:],sigmays[i,:])
    xplot=linspace(-5000.0,300000.0,100)
    yplot = slope * xplot + intercept
    plot(xplot, yplot, color='red', lw = 2, ls = '--')
    tslope = slope * 100.0 * 50000.0
    text(10000.0,1.082,"Y Slope = %.2f %% per 50K e-, Intercept = %.3f"%(tslope,intercept), fontsize=10)

    # Now plot the corrected data
    scatter(delete(corr_imaxs[i,:],mask), delete(corr_sigmaxs[i,:],mask), color = 'cyan', lw = 2, marker = 'x',label = 'Corrected Sigma-x')
    scatter(delete(corr_imaxs[i,:],mask), delete(corr_sigmays[i,:],mask), color = 'magenta', lw = 2, marker = 'x', label = 'Corrected Sigma-y')

    slope, intercept, r_value, p_value, std_err = stats.linregress(corr_imaxs[i,:],corr_sigmaxs[i,:])
    xplot=linspace(-5000.0,300000.0,100)
    yplot = slope * xplot + intercept
    plot(xplot, yplot, color='cyan', lw = 2, ls = '--')
    tslope = slope * 100.0 * 50000.0
    text(10000.0,1.074,"Corrected X Slope = %.2f %% per 50K e-, Intercept = %.3f"%(tslope,intercept), fontsize=10)

    slope, intercept, r_value, p_value, std_err = stats.linregress(corr_imaxs[i,:],corr_sigmays[i,:])
    xplot=linspace(-5000.0,300000.0,100)
    yplot = slope * xplot + intercept
    plot(xplot, yplot, color='magenta', lw = 2, ls = '--')
    tslope = slope * 100.0 * 50000.0
    text(10000.0,1.066,"Corrected Y Slope = %.2f %% per 50K e-, Intercept = %.3f"%(tslope,intercept), fontsize=10)

    xlim(0.0,150000.0)
    xticks([0,50000,100000])
    ylim(1.00,1.10)
    xlabel('Central Peak(electrons)')
    ylabel('Sigma (Pixels)')
    legend(loc= 'lower right',fontsize = 9)#, bbox_to_anchor=[1.8,-0.1])
    savefig(datadir+'Forward_Model_Central_CSteps_7_%s_07Jun18.pdf'%extname)


In [ ]:
figure(figsize=(16,8))
title("Flux Lost in BF Kernel Correction")
for i, segmentnumber in enumerate([1]):
    hdr=pf.getheader(zfilelist[0],segmentnumber)
    extname = hdr['EXTNAME']
    flux_loss = (totelectrons[i,:] - corr_totelectrons[i,:]) / totelectrons[i,:] * 100.0
    print extname, flux_loss.max(), flux_loss.min()
    plot(imaxs[i,:], flux_loss, label = extname)

    xlim(0.0,300000.0)
    xticks([0,100000,200000])
    ylim(-0.5,0.1)
    xlabel('Central Peak(electrons)')
    ylabel('Flux Loss (%)')
legend(loc='lower right')
savefig(datadir+'Flux_Loss_Model_Rot_CSteps_7_05Jun18.png')
